In [1]:
%load_ext autoreload
%autoreload 2
import warnings 
warnings.simplefilter(action='ignore')
!python --version

Python 3.6.9 :: Anaconda, Inc.


In [2]:
%%writefile conda_dependencies.yml
dependencies:
    - python=3.6.9
    - numpy=1.18.*
    - pandas=0.25.*
    - pip:
        - azureml-defaults==1.34.0
        - confuse==1.6.0
        - icecream==2.1.1
        - plotly==5.3.1
        - scikit-surprise==1.1.1
        - kaggle==1.5.12

Overwriting conda_dependencies.yml


In [3]:
%%writefile requirements.txt
confuse==1.6.0
icecream==2.1.1
plotly==5.3.1
kaleido==0.2.1
scikit-surprise==1.1.1
kaggle==1.5.12

Overwriting requirements.txt


### Leverage Kaggle API for Subreddit Dataset

In [5]:
!pip3 install -r requirements.txt

In [3]:
import json
from kaggle.api.kaggle_api_extended import KaggleApi

with open('kaggle.json') as k_f:
    kaggle_auth = json.load(k_f)
k_api = KaggleApi()
k_api._load_config(config_data=kaggle_auth)

In [6]:
from icecream import ic
import os

data_folder_name = 'data'
if not (os.path.exists(data_folder_name)):
    os.mkdir(data_folder_name)
os.chdir(data_folder_name)

In [7]:
reference = k_api.datasets_list(search='Subreddit Recommender')[0]['ref']
ic(reference);
k_api.dataset_download_files(reference)

ic| reference: 'timschaum/subreddit-recommender'


In [42]:
from zipfile import ZipFile

zf_name = reference.split('/')[1] + '.zip'
zf = ZipFile(zf_name)
zf.extractall()
zf.close()

In [7]:
original_dir = os.path.dirname(os.getcwd())
os.chdir(original_dir)